In [1]:
import numpy as np
import pandas as pd 
# import tensorflow as tf 
import sklearn as sk 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from scipy import stats
from scipy.stats import norm, skew 
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import PolynomialFeatures

from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings("ignore")

In [8]:
dataset_train = pd.read_csv('train.csv')

X = dataset_train.iloc[:, 1:12]
n = X.shape[0]
# print(X.head(3))
Y = dataset_train.iloc[:, 12]
# test train split
# # =============================================================================
dataset_test = pd.read_csv('test.csv')
Xtest = dataset_test.iloc[:, 1:12]

X_all = pd.concat([X,Xtest], axis=0)

X_all['Soil_Type'] = X_all['Soil_Type'].astype('category') 
X_all = pd.get_dummies(X_all)



scaler = StandardScaler().fit(np.array(X.iloc[:,:9]))
X_std = scaler.transform(np.array(X.iloc[:,:9]))
X_std = pd.DataFrame(X_std)

X_std_full = pd.concat([X_std,X_all.iloc[:n,10::]],axis = 1)
print(list(X_std_full))

X_train, X_test, y_train, y_test = train_test_split(X_std_full, Y, test_size=0.1, random_state=42)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 'Soil_Type_2702', 'Soil_Type_2703', 'Soil_Type_2704', 'Soil_Type_2705', 'Soil_Type_2706', 'Soil_Type_2717', 'Soil_Type_4703', 'Soil_Type_4704', 'Soil_Type_4758', 'Soil_Type_5101', 'Soil_Type_6101', 'Soil_Type_6102', 'Soil_Type_7102', 'Soil_Type_7756', 'Soil_Type_7757']


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_std_full, Y, test_size=0.15, random_state=42)

cross validation

In [4]:
from sklearn.model_selection import GridSearchCV # Search over specified parameter values for an estimator.
from sklearn.model_selection import RandomizedSearchCV # Search over specified parameter values for an estimator.
from sklearn.model_selection import ShuffleSplit # Random permutation cross-validator
from sklearn.svm import SVC
from AUC import AUC
classifier = SVC(kernel = 'rbf', random_state=42)
cv_sets = ShuffleSplit(random_state = 4) # shuffling our data for cross-validation
parameters = {'gamma':(np.arange(1,40)/20).tolist(),
			  'degree':(np.arange(1,40)/2).tolist(),
			  'C': (np.arange(1,40)/5).tolist(),
			  'coef0' :(np.arange(1,40)/20).tolist()}
scorer = make_scorer(AUC,greater_is_better =True)
n_iter_search = 10
grid_obj = RandomizedSearchCV(classifier, 
                              parameters, 
                              n_iter = n_iter_search, 
                              scoring = scorer, 
                              cv = cv_sets,
                              random_state= 99)
grid_fit = grid_obj.fit(X_train, y_train)
rf_opt = grid_fit.best_estimator_
print(grid_fit.best_estimator_)

SVC(C=6.4, cache_size=200, class_weight=None, coef0=0.7,
  decision_function_shape='ovr', degree=1.5, gamma=0.9, kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)


In [7]:
dataset_test = pd.read_csv('test.csv')

Xtest = dataset_test.iloc[:, 1:12]
# print(Xtest.head(3))
Xtest['Soil_Type'] = Xtest['Soil_Type'].astype('category') 

scaler = StandardScaler().fit(np.array(X.iloc[:,[0,1,2,3,4,5,6,7,8,9]]))
Xtest_std = scaler.transform(np.array(Xtest.iloc[:,[0,1,2,3,4,5,6,7,8,9]]))
Xtest_std = pd.DataFrame(Xtest_std)
Xtest_std_full = pd.concat([Xtest_std,X_all.iloc[n::,10::]],axis = 1)

print(Xtest_std_full.head(3))


#ids = dataset_test['ID']
#predictions = svclass(X_std_full,Y,Xtest_std_full)

# output = pd.DataFrame({ 'ID' : ids, 'From_Cache_la_Poudre': predictions })
# output.to_csv('svm_submission.csv', index = False)

          0         1         2         3         4         5         6  \
0  1.518814 -0.393620  0.362423  0.222200  0.803259  1.637816  1.153551   
1  0.575510 -1.377395  1.253873  0.166062  0.752148  0.311122 -0.325161   
2 -1.785298  1.286224  0.139560 -1.265448 -0.866373 -0.565211 -1.138453   

          7         8         9       ...        Soil_Type_2717  \
0  0.190339 -1.030104  3.421162       ...                     0   
1 -2.186893 -1.068270 -0.001647       ...                     0   
2  0.010246  1.107186 -0.589732       ...                     0   

   Soil_Type_4703  Soil_Type_4704  Soil_Type_4758  Soil_Type_5101  \
0               0               0               0               0   
1               1               0               0               0   
2               0               0               0               0   

   Soil_Type_6101  Soil_Type_6102  Soil_Type_7102  Soil_Type_7756  \
0               0               0               0               0   
1              